In [19]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/My Drive/589MiniProject/")
%cd /content/drive/My Drive/589MiniProject/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1yyh1nEBAISidzfrj3Aql57NwD7xFpa20/589MiniProject


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 75

print('using device:', device)

using device: cpu


In [9]:
train_df = pd.read_csv('Data/MetaData/train.csv')
test_df = pd.read_csv('Data/MetaData/test.csv')
val_df = pd.read_csv('Data/MetaData/valid.csv')

train_df['img_path'] = os.getcwd() + '/Data/Train' + train_df['Id'] + '.jpg'
test_df['img_path'] = os.getcwd() + '/Data/Test' + test_df['Id'] + '.jpg'
val_df['img_path'] = os.getcwd() + '/Data/Valid' + val_df['Id'] + '.jpg'
test_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,img_path
0,dd0b6b124a9de66d3f55822b1e2f50fa,0,1,1,1,0,1,0,0,0,0,0,0,35,/content/drive/.shortcut-targets-by-id/1yyh1nE...
1,d98e0578566c1f7c791aab11b02726e8,0,1,1,1,0,0,0,0,0,0,0,0,21,/content/drive/.shortcut-targets-by-id/1yyh1nE...
2,00d1cb2ec8b263ae076ff95cae513a88,0,1,1,1,0,0,0,0,0,0,0,0,31,/content/drive/.shortcut-targets-by-id/1yyh1nE...
3,facb4a4f9de4b590c95a06e3f39c4523,0,0,1,0,0,0,1,0,0,0,0,0,47,/content/drive/.shortcut-targets-by-id/1yyh1nE...
4,5e159ec567ffb4a8121c235305557f96,0,0,0,0,0,0,0,0,0,0,0,0,25,/content/drive/.shortcut-targets-by-id/1yyh1nE...


In [17]:
data_dir = "Data/"
ITRAIN = "Train"
IVAL = "Valid"
ITEST = "Test"
# MTRAIN =
# MVAL =
# MTEST =

In [21]:
data_transforms = {ITRAIN : transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]),
IVAL : transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]),
ITEST : transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])}

In [20]:
print(os.getcwd())
image_datasets = {
    x:datasets.ImageFolder(os.path.join(data_dir,x), transform = data_transforms[x]) for x in [ITRAIN, IVAL, ITEST]
}
data_loaders = {
    x:DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=4) for x in [ITRAIN, IVAL, ITEST]
}
print(data_loaders[ITEST])

/content/drive/.shortcut-targets-by-id/1yyh1nEBAISidzfrj3Aql57NwD7xFpa20/589MiniProject


FileNotFoundError: ignored

In [ ]:
loss_history = []

In [ ]:
def check_squared_error(loader,model):
    print("checking squared error difference")
    squared_error = 0
    num_samples = 0
    y_true, y_preds = [],[]
    model.eval()  # set model to evaluation mode
    label_loader = ######
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred = model(x)
            squared_error += torch.sum(torch.square(y_pred - y))
            num_samples += preds.size(0)
        mse = squared_error / num_samples
        print('mse:',mse)
    return mse

In [ ]:
def train_model(model, optimizer, epochs=1):
    """
    Train a model on Melanoma dataset using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    best_mse = float('inf')
    for e in range(epochs):
        print('epoch:',e)
        loss_history.append([]) 
        for t, (x, y) in enumerate(data_loaders[MTRAIN]):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            img = []
            for id in data_loaders[MTRAIN][x][:,0]:
                img.append(data_loaders)
            scores = model(img,x)
            
            loss = F.mse_loss(scores,y)
            loss_history[e].append(loss.item())
            # print('cross-entropy loss:',loss)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            torch.cuda.empty_cache()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                mse = check_squared_error(data_loaders[VAL], model)
                if mse > best_mse:
                  torch.save(model.state_dict(), "./Models/cnn.pt")
                  best_mse = mse
                torch.cuda.empty_cache()

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cnn = models.vgg16(pretrained=False, aux_logits=False)
        self.cnn.fc = nn.Linear(
            self.cnn.fc.in_features, 4096)
        
        self.fc1 = nn.Linear(4096 + 14, 3000)
        self.fc2 = nn.Linear(3000, 1)
        
    def forward(self, image, meta_data):
        x1 = self.cnn(image)
        x2 = meta_data
        
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
